In [29]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [30]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.16.2
TFX version: 1.16.0


In [31]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX of HAR
_taxi_root = os.path.join(_tfx_root, 'examples/har_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/har_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [32]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/COEN498-691-PROJECT/ML_project/refs/heads/main/data/processed/COEN498-691_HAR_preprocessed_dataset.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-datavl0yvf8c/data.csv',
 <http.client.HTTPMessage at 0x18804c850>)

In [33]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

In [34]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-datavl0yvf8c/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [35]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-interactive-2025-12-05T19_19_54.779121-mye8ktxd/CsvExampleGen/examples/1


In [36]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Gx"
    value {
      float_list {
        value: 0.06556067615747452
      }
    }
  }
  feature {
    key: "Gx_angle"
    value {
      float_list {
        value: 1.5051885843276978
      }
    }
  }
  feature {
    key: "Gy"
    value {
      float_list {
        value: -0.9814617037773132
      }
    }
  }
  feature {
    key: "Gy_angle"
    value {
      float_list {
        value: 2.9487411975860596
      }
    }
  }
  feature {
    key: "Gz"
    value {
      float_list {
        value: 0.1800963282585144
      }
    }
  }
  feature {
    key: "Gz_angle"
    value {
      float_list {
        value: 1.3897119760513306
      }
    }
  }
  feature {
    key: "activity_id"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "axG_mean"
    value {
      float_list {
        value: 0.06839167326688766
      }
    }
  }
  feature {
    key: "ax_kurt"
    value {
      float_list {
        value: 1.735067367553711
   

2025-12-05 19:20:14.646642: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [37]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-interactive-2025-12-05T19_19_54.779121-mye8ktxd/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-interactive-2025-12-05T19_19_54.779121-mye8ktxd/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [38]:
context.show(statistics_gen.outputs['statistics'])

SchemaGen

In [39]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-interactive-2025-12-05T19_19_54.779121-mye8ktxd/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [40]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Gx',FLOAT,required,single,-
'Gx_angle',FLOAT,required,single,-
'Gy',FLOAT,required,single,-
'Gy_angle',FLOAT,required,single,-
'Gz',FLOAT,required,single,-
'Gz_angle',FLOAT,required,single,-
'activity_id',INT,required,single,-
'axG_mean',FLOAT,required,single,-
'ax_kurt',FLOAT,required,single,-


,Values
Domain,
'participant_id',"'LL', 'VV', 'YL', 'YT', 'ZM'"


ExampleValidator

In [41]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-interactive-2025-12-05T19_19_54.779121-mye8ktxd/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-interactive-2025-12-05T19_19_54.779121-mye8ktxd/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [42]:
context.show(example_validator.outputs['anomalies'])

Transform

In [43]:
_har_constants_module_file = 'har_constants.py'

In [44]:
%%writefile {_har_constants_module_file}

NUMERICAL_FEATURES = ['ax_mean','ax_std','ax_max','ax_min','ax_range',
            'ax_skew','ax_kurt','ax_zcr',
            'ay_mean','ay_std','ay_max','ay_min','ay_range',
            'ay_skew','ay_kurt','ay_zcr',
            'az_mean','az_std','az_max','az_min','az_range',
            'az_skew','az_kurt','az_zcr',
            'sma','corr_axy','corr_axz','corr_ayz',
            'axG_mean','ayG_mean','azG_mean',
            'Gx','Gy','Gz','Gx_angle','Gy_angle','Gz_angle'
]

# BUCKET_FEATURES = [
#     ...
# ]
# # Number of buckets used by tf.transform for encoding each feature.
# FEATURE_BUCKET_COUNT = 10

CATEGORICAL_STRING_FEATURES = [
    'participant_id'
]

# Number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized categorical are hashed.
# no bucket
OOV_SIZE = 0

# Keys
LABEL_KEY = 'activity_id'


def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

Overwriting har_constants.py


In [45]:
_har_transform_module_file = 'har_transform.py'

In [46]:
%%writefile {_har_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as har_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import har_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(har_constants)

_NUMERICAL_FEATURES = har_constants.NUMERICAL_FEATURES
# _BUCKET_FEATURES = har_constants.BUCKET_FEATURES
# _FEATURE_BUCKET_COUNT = har_constants.FEATURE_BUCKET_COUNT
# _CATEGORICAL_NUMERICAL_FEATURES = har_constants.CATEGORICAL_NUMERICAL_FEATURES
_CATEGORICAL_STRING_FEATURES = har_constants.CATEGORICAL_STRING_FEATURES
_VOCAB_SIZE = har_constants.VOCAB_SIZE
_OOV_SIZE = har_constants.OOV_SIZE
_LABEL_KEY = har_constants.LABEL_KEY


def _make_one_hot(x, key):
  """Make a one-hot tensor to encode categorical features.
  Args:
    X: A dense tensor
    key: A string key for the feature in the input
  Returns:
    A dense one-hot tensor as a float list
  """
  integerized = tft.compute_and_apply_vocabulary(x,
          top_k=_VOCAB_SIZE,
          num_oov_buckets=_OOV_SIZE,
          vocab_filename=key, name=key)
  depth = (
      tft.experimental.get_vocabulary_size_by_name(key) + _OOV_SIZE)
  one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(depth, tf.int32),
      on_value=1.0,
      off_value=0.0)
  return tf.reshape(one_hot_encoded, [-1, depth])


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[har_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

#   for key in _BUCKET_FEATURES:
#     outputs[har_constants.t_name(key)] = tf.cast(tft.bucketize(
#             _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT, name=key),
#             dtype=tf.float32)

  for key in _CATEGORICAL_STRING_FEATURES:
    outputs[har_constants.t_name(key)] = _make_one_hot(_fill_in_missing(inputs[key]), key)

#   for key in _CATEGORICAL_NUMERICAL_FEATURES:
#     outputs[har_constants.t_name(key)] = _make_one_hot(tf.strings.strip(
#         tf.strings.as_string(_fill_in_missing(inputs[key]))), key)


  return outputs

Overwriting har_transform.py


In [48]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_har_transform_module_file))
context.run(transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/Users/laura/VSCodeProj/ML_project/tfx/har_transform.py' (including modules: ['har_transform', 'har_constants']).
INFO:absl:User module package has hash fingerprint version 686e8bd7115ee981bf848cdd9af33d0ae65aa4a397f4b5ddb4427f9d6a628f12.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpf46q4o0e/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpa7p1l8om', '--dist-dir', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpfn1buxrt']
INFO:absl:Successfully built user code wheel distribution at '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tfx-interactive-2025-12-05T19_19_54.779121-mye8ktxd/_wheels/tfx_user_code_Transform-0.0+686e8bd7115ee981bf848cdd9af33d0ae65aa4a397f4b5ddb4427f9d6a628f12-py3-none-any.whl'; target user module is 'har_transform'.
INFO:absl:Full user module path is 'har_transform@/v

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying har_transform.py -> build/lib
copying har_constants.py -> build/lib
installing to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpa7p1l8om
running install
running install_lib
copying build/lib/har_transform.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpa7p1l8om
copying build/lib/har_constants.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpa7p1l8om
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.e

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [49]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_componen

In [50]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [51]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Gx_angle_xf"
    value {
      float_list {
        value: 0.20972275733947754
      }
    }
  }
  feature {
    key: "Gx_xf"
    value {
      float_list {
        value: -0.17126907408237457
      }
    }
  }
  feature {
    key: "Gy_angle_xf"
    value {
      float_list {
        value: 0.6923685669898987
      }
    }
  }
  feature {
    key: "Gy_xf"
    value {
      float_list {
        value: -0.63109290599823
      }
    }
  }
  feature {
    key: "Gz_angle_xf"
    value {
      float_list {
        value: 0.5154333114624023
      }
    }
  }
  feature {
    key: "Gz_xf"
    value {
      float_list {
        value: -0.508447527885437
      }
    }
  }
  feature {
    key: "axG_mean_xf"
    value {
      float_list {
        value: -0.1481943279504776
      }
    }
  }
  feature {
    key: "ax_kurt_xf"
    value {
      float_list {
        value: 1.553745150566101
      }
    }
  }
  feature {
    key: "ax_max_xf"
    value {
      float_list 

2025-12-05 19:22:57.856073: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Trainer

In [ ]:
_har_trainer_module_file = 'har_trainer.py'